In [1]:
!pip install opendatasets
# !pip install -U spacy -qq
!python -m spacy download en_core_web_sm
#!pip install xgboost


2023-09-11 09:11:11.558985: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-11 09:11:14.186959: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 32.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [2]:
import opendatasets as od
import pandas as pd
import chardet
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
#from sklearn.model_selection import RepeatedStratifiedKFold
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
import spacy
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
import numpy as np
import re
import os
from google.colab import drive
from bs4 import BeautifulSoup as bs
from pathlib import Path
from sklearn.metrics import classification_report
from sklearn.metrics import RocCurveDisplay
import string
from textblob import TextBlob
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingRandomSearchCV
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV
import collections

In [3]:
od.download("https://www.kaggle.com/datasets/uciml/sms-spam-collection-dataset")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: shremuk
Your Kaggle Key: ··········


100%|██████████| 211k/211k [00:00<00:00, 17.4MB/s]

# **Reading the file**

In [4]:
file =('sms-spam-collection-dataset/spam.csv')
df = pd.read_csv('sms-spam-collection-dataset/spam.csv', encoding = 'Windows-1252') # delimiter = ',', encoding="latin1"
df.rename(columns = {'v1':'labels', 'v2': 'message'}, inplace = True)
df.drop({'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'}, axis = 1, inplace = True)
df = df[['message', 'labels']]
df.labels.replace({'spam': 1, 'ham': 0}, inplace = True)
df['message'] = [bs(text).getText() for text in df.message]

<ipython-input-4-8e141cc8bd3c>:7: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  df['message'] = [bs(text).getText() for text in df.message]


In [5]:
df.head()

,message,labels
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0


In [6]:
df.isna().sum()

message    0
labels     0
dtype: int64

In [7]:
df.shape[0]

5572

**% Ham and spam in the dataset**

In [8]:
round(df.labels.value_counts(normalize = True) * 100, 3)

0    86.594
1    13.406
Name: labels, dtype: float64

# **Choice of accuracy metric**

**Since importance needs to be given to the correct classification for spam(class 1) hence I'd go ahead with recall score. Will try to maximize recall for this dataset. However, will also check the f1 score so that at the end, it balances precision and recall**

**I chose Pipeline 3 because it givves me a recall of around 60% with an f1 score of 72% for class 1(SPAM) whereas the other 2 are overfitting the data.**

## **Feature Engineering**

In [9]:
df.message = df.message.str.replace(r"^.+@.+\.{2,}$", '')
df.message = df.message.str.replace(r'\s*https?://\S+(\s+|$)', '').str.strip() #apply(lambda x: re.split('https:\/\/.*', str(x))[0])
df.message = df.message.str.replace('[{}]'.format(string.punctuation), '')
df.message = df.message.str.replace(r'[\"\\...\\\n]', '')
df.message = df.message.str.replace(r'[...]', ' ')

<ipython-input-9-eb6e7a6e2216>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df.message = df.message.str.replace(r"^.+@.+\.{2,}$", '')
<ipython-input-9-eb6e7a6e2216>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df.message = df.message.str.replace(r'\s*https?://\S+(\s+|$)', '').str.strip() #apply(lambda x: re.split('https:\/\/.*', str(x))[0])
<ipython-input-9-eb6e7a6e2216>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df.message = df.message.str.replace('[{}]'.format(string.punctuation), '')
<ipython-input-9-eb6e7a6e2216>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df.message = df.message.str.replace(r'[\"\\...\\\n]', '')
<ipython-input-9-eb6e7a6e2216>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df.message = df.message.st

In [10]:
df['Word_count'] = df.message.str.split().str.len()
df['Character_count'] = df.message.str.len()
df['Character_count_wo_space'] = df.message.str.replace('\s', '', regex=True).str.len()
df['average_word_length'] = df.message.apply(lambda x: np.mean([len(w) for w in x.split()])).round(3)
df['count_of_digits'] = df.message.str.split().str.isdigit().sum()
df['extract_nos'] = df.message.str.findall(r'(\d+)').str.len()

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,


In [11]:
nlp = spacy.load("en_core_web_sm")
def count_noun(x):
  res = [token.pos_ for token in nlp(x)]
  return collections.Counter(res)['NOUN']

df['noun'] = df.message.map(count_noun)

In [12]:
# def get_adjectives(text):
#   blob = TextBlob(text)
#   return [ word for (word,tag) in blob.tags if tag == "JJ"]

# df['adjectives'] = df.message.apply(get_adjectives)

In [13]:
df.head()

,message,labels,Word_count,Character_count,Character_count_wo_space,average_word_length,count_of_digits,extract_nos,noun
0,Go until jurong point crazy Available only in ...,0,20,102,83,4.150,0.0,0,3
1,Ok lar Joking wif u oni,0,6,23,18,3.000,0.0,0,2
2,Free entry in 2 a wkly comp to win FA Cup fina...,1,28,149,122,4.357,0.0,6,8
3,U dun say so early hor U c already then say,0,11,43,33,3.000,0.0,0,5
4,Nah I dont think he goes to usf he lives aroun...,0,13,59,47,3.615,0.0,0,0


## **Pipeline 3: Combine Manual Features and TfID vectors**

#### **Creating the dataframe with Sparse embedding and feature engineering**

Splitting into train and text

In [15]:
X_final = df.drop('labels', axis = 1)
y_final = df['labels']
X_train_final, X_test_final, y_train_final, y_test_final = train_test_split(X_final, y_final, test_size = 0.3, random_state = 66)

### **Picking up 200 samples**

In [16]:
X_final_sample = X_train_final.sample(n = 200, replace=True, random_state=30)
y_final_sample = y_train_final.sample(n = 200, replace=True, random_state=30)
print(X_final_sample.shape, y_final_sample.shape)

(200, 8) (200,)


### **Creating Pipeline to run the samples**

In [20]:
pipe3 = Pipeline([('vectorizer', TfidfVectorizer(analyzer='word', token_pattern=r"[\S]+", ngram_range = (1, 2))), \
                       ('model', XGBClassifier(n_estimators=100, learning_rate = 0.1))])

In [21]:
params = {'model__max_depth': range(1, 10, 1), 'model__n_estimators':[100, 300, 500], 'model__eta': [0.001, 0.01, 0.05, 0.1, 1]}
gsfinal = GridSearchCV(estimator = pipe3, param_grid = params, scoring = 'roc_auc', cv = 5)
model_final = gsfinal.fit(X_final_sample, y_final_sample)
print(f'Best cross-validation score: {model_final.best_score_:.2f}')
print("\nBest parameters: ", model_final.best_params_)
print("\nBest estimator: ", model_final.best_estimator_)

Streaming output truncated to the last 5000 lines.
    return self._score(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 390, in _score
    y_pred = self._select_proba_binary(y_pred, clf.classes_)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 198, in _select_proba_binary
    raise ValueError(err_msg)
ValueError: Got predict_proba of shape (8, 20), but need classifier with two classes for roc_auc_score scoring

  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 373, in _score
    y_pred = method_caller(clf, "decision_function", X)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 73, in _cached_call
    

KeyboardInterrupt: ignored

### **Fitting the best params on the whole dataset**

In [ ]:
pipe4 = Pipeline([('vectorizer', TfidfVectorizer(analyzer='word', token_pattern=r"[\S]+", ngram_range = (1, 2))), \
                  ('model', XGBClassifier(n_estimators=500, learning_rate = 0.001, max_depth = 2))])
result = pipe4.fit(X_train_final, y_train_final)
y_pred_final = result.predict(X_test_final)

In [ ]:
print(classification_report(y_test_final, y_pred_final))

In [ ]:
RocCurveDisplay.from_estimator(pipe4, X_train_final, y_train_final)

In [ ]:
RocCurveDisplay.from_estimator(pipe4, X_test_final, y_test_final)